### Load packages

In [1]:
import os, sys
import numpy as np
import pandas as pd
from glob import glob 
from skimage.io import imread, imsave
import zarr
import napari

### Start Napari viewer

In [2]:
%gui qt
napari.gui_qt()
viewer = napari.Viewer(ndisplay=3)

### Specify directory to data files

In [3]:
roi_dir='/test/LHA3_R3_medium/spots/roi.csv'
lb_dir='/test/LHA3_R3_medium/segmentation/LHA3_R3_medium-c2.tif'
spot_dir='/test/LHA3_R5_medium/spots/LHA3_R5_medium-to-LHA3_R3_medium/merged_points_c3_warped.txt'
imdir="/test/LHA3_R5_medium/registration/LHA3_R5_medium-to-LHA3_R3_medium/warped/"

### Specify ROI ID, block size and image channel 

In [24]:
#Input information here: cell ID, image block size, channels
cell_id=3 

#define the size of the block to display around the ROI
dx,dy,dz=[100,100,50]  # in pixel 
channel_1='c3'
channel_2=''

### Load data

In [25]:
roi=pd.read_csv(roi_dir,sep=',', index_col=1)
lb=imread(lb_dir)
z=roi.loc[cell_id,'z']*2/0.84 # convert to pixel 
y=roi.loc[cell_id,'y']*2/0.92 # convert to pixel 
x=roi.loc[cell_id,'x']*2/0.92 # convert to pixel 
Z=int(z)
Y=int(y)
X=int(x)

#Crop image to show the target cell and surrounding area
lb_sub=lb[np.maximum(Z-dz,0):Z+dz,np.maximum(Y-dy,0):Y+dy,np.maximum(X-dx,0):X+dx]
#Target Cell ID
z0=z-Z+dz
y0=y-Y+dy
x0=x-X+dx
a=np.array([[z0],[y0],[x0]])
a=np.transpose(a)

### Display segmentation mask, image data and the position of the specified ROI

In [26]:
# get n5 image data for channel 1
im = zarr.open(store=zarr.N5Store(imdir), mode='r')
img_1 = im[channel_1+'/s2'][Z-dz:Z+dz,Y-dy:Y+dy,X-dx:X+dx]
viewer.add_image(img_1.astype('float32'),colormap='green',blending='additive')
#get n5 image data for channel 2
if channel_2!='':
    im = zarr.open(store=zarr.N5Store(imdir), mode='r')   
    img_2 = im[channel_2+'/s2'][Z-dz:Z+dz,Y-dy:Y+dy,X-dx:X+dx]
    viewer.add_image(img_2.astype('float32'),colormap='gray',blending='additive')
zo=z-Z+dz
yo=y-Y+dy
xo=x-X+dx
a=np.array([[zo],[yo],[xo]])
a=np.transpose(a)
viewer.add_points(a, size=5, face_color='red',edge_color='red',name='cell_centroid_position')
lb_sub=lb[Z-dz:Z+dz,Y-dy:Y+dy,X-dx:X+dx]
viewer.add_labels(lb_sub.astype('uint32'),name='labels',blending='additive')

<Labels layer 'labels' at 0x1c2c317110>

### Spot detection can also be visualized 

In [30]:
spot=np.loadtxt(spot_dir,delimiter=' ')
vox=[0.92,0.92,0.84]  #current image voxel size
spot[:,:3]=spot[:,:3]/vox

### Display spot detection in selected image block

In [32]:

b=spot[np.logical_and(spot[:,0]<=X+dx,spot[:,0]>X-dx)]
b=b[np.logical_and(b[:,1]<=Y+dy,b[:,1]>Y-dy)]
b=b[np.logical_and(b[:,2]<=Z+dz,b[:,2]>Z-dz)]
bx=b[:,0]-X+dx
by=b[:,1]-Y+dy
bz=b[:,2]-Z+dz
point=np.stack((bz,by,bx),axis=-1)
viewer.add_points(point, size=1, face_color='red',edge_color='red',name='detected_spots')

<Points layer 'detected_spots' at 0x1c2f0a2ed0>